In [1]:
import numpy as np
import pandas as pd

# Data Structures
## `Series`

### What is a Series?

A **pandas Series** is a **one-dimensional labeled array** that can hold any data type — integers, floats, strings, Python objects, and more.

Each element has:

* a **value**
* an **index label**

```python
s = pd.Series(data, index=index)
```

Like a NumPy array, a pandas Series has a single dtype.

### Creating a Series

#### From a NumPy array (`ndarray`)

If `data` is an array, the index must be the same length as the array.
If no index is given, pandas creates a default one: `[0, 1, 2, …]`.

In [2]:
s = pd.Series(np.random.randn(5), index=["a", "b", "c", "d", "e"])
print("Series:\n", s, end="\n\n")
print("Indexes:", s.index)  
print("Data type:", s.dtype)

Series:
 a    1.087721
b   -0.302828
c    0.830606
d    0.835890
e   -0.919755
dtype: float64

Indexes: Index(['a', 'b', 'c', 'd', 'e'], dtype='object')
Data type: float64


Without an index:

In [3]:
s_def = pd.Series(np.random.randn(5))

print("Series with Default Indexes:\n", s_def, end="\n\n")
print("Default Indexes:", s_def.index)

Series with Default Indexes:
 0   -0.818179
1    0.777865
2    0.939725
3    0.732935
4   -0.758164
dtype: float64

Default Indexes: RangeIndex(start=0, stop=5, step=1)


#### From a dictionary

When created from a dictionary, the keys become the index, and the values become the data.

In [4]:
d = {"b": 1, "a": 0, "c": 2}
pd.Series(d)

b    1
a    0
c    2
dtype: int64

You can also specify a custom index:

In [5]:
d = {"a": 0.0, "b": 1.0, "c": 2.0}
pd.Series(d, index=["b", "c", "d", "a"])

b    1.0
c    2.0
d    NaN
a    0.0
dtype: float64

> ⚠️ **Note:** Missing labels produce `NaN` (Not a Number) — pandas’ standard marker for missing data.

#### From a scalar value

If you pass a single value, it will be repeated for every index entry.

In [6]:
pd.Series(5.0, index=["a", "b", "c", "d", "e"])

a    5.0
b    5.0
c    5.0
d    5.0
e    5.0
dtype: float64

### Series behaves like a NumPy array

Series acts very similarly to a ndarray and is a valid argument to most NumPy functions. 
However, slicing also affects the index.

In [7]:
print("First element:", s.iloc[0], end="\n\n")
print("First three elements:\n", s.iloc[:3], end="\n\n")
print("Boolean indexing:\n", s[s > s.median()], end="\n\n")
print("Element-wise exponential:\n", np.exp(s))

First element: 1.0877210290098545

First three elements:
 a    1.087721
b   -0.302828
c    0.830606
dtype: float64

Boolean indexing:
 a    1.087721
d    0.835890
dtype: float64

Element-wise exponential:
 a    2.967504
b    0.738726
c    2.294708
d    2.306866
e    0.398617
dtype: float64


#### Extracting the underlying data

In [8]:
s.to_numpy()

array([ 1.08772103, -0.30282756,  0.83060566,  0.8358897 , -0.91975523])

### Series behaves like a dict

You can access or set elements by label — just like in a dictionary.

In [9]:
print("Access s[\"a\"] by label:", s["a"]) # Access by label
s["e"] = 12.0 # Modify value

Access s["a"] by label: 1.0877210290098545


Check if a label exists:

In [10]:
print("Check if 'e' is in index:", "e" in s)   # True
print("Check if 'f' is in index:", "f" in s)

Check if 'e' is in index: True
Check if 'f' is in index: False


If you try to access a missing label directly...

In [11]:
try:
    s["f"]
except KeyError as e:
    print("KeyError:", e)

KeyError: 'f'


Use `.get()` to avoid errors:

In [12]:
print(s.get("f"))
print(s.get("f", np.nan))

None
nan


### Vectorized operations and label alignment

Operations between Series are **element-wise** and automatically **aligned by index labels**.

In [13]:
print(s)
print("s + s\n", s + s, end="\n\n")
print("s * 2\n", s * 2, end="\n\n")
print("np.exp(s)\n", np.exp(s), end="\n\n")

a     1.087721
b    -0.302828
c     0.830606
d     0.835890
e    12.000000
dtype: float64
s + s
 a     2.175442
b    -0.605655
c     1.661211
d     1.671779
e    24.000000
dtype: float64

s * 2
 a     2.175442
b    -0.605655
c     1.661211
d     1.671779
e    24.000000
dtype: float64

np.exp(s)
 a         2.967504
b         0.738726
c         2.294708
d         2.306866
e    162754.791419
dtype: float64



When combining two Series with different indexes, pandas aligns by label and fills missing values with `NaN`.

In [14]:
s.iloc[1:] + s.iloc[:-1]

a         NaN
b   -0.605655
c    1.661211
d    1.671779
e         NaN
dtype: float64

> **Note:** pandas chooses to return the **union** of index labels to preserve information.
> Use `.dropna()` to remove missing values if needed.

### The `name` Attribute

Each Series has a `name` that can help identify it (e.g., when part of a DataFrame).

In [15]:
s = pd.Series(np.random.randn(5), name="something")
s.name

'something'

Rename it:

In [16]:
s2 = s.rename("different")
s2.name

'different'

> ! `s` and `s2` are separate objects.

## `DataFrame`

### What is a DataFrame?

A **pandas DataFrame** is a **two-dimensional labeled data structure** with columns that can have different data types.
You can think of it as:

* a spreadsheet 🧾
* a SQL table 🗃️
* or a **dictionary of Series objects**

DataFrames are the most commonly used object in pandas.

### Creating a DataFrame

You can build a DataFrame from many kinds of input:

* Dict of 1D arrays / lists / dicts / Series
* 2D NumPy array
* Structured (record) ndarray
* Another Series or DataFrame
* List of dicts, namedtuples, or dataclasses

Optionally, you can specify:

* `index` → row labels
* `columns` → column labels

If omitted, pandas creates them automatically.

#### From a dict of Series or dicts

Each key becomes a column name.
The resulting index is the **union** of all Series indexes.

In [17]:
d = {
    "one": pd.Series([1.0, 2.0, 3.0], index=["a", "b", "c"]),
    "two": pd.Series([1.0, 2.0, 3.0, 4.0], index=["a", "b", "c", "d"]),
}

df = pd.DataFrame(d)
print(df)

   one  two
a  1.0  1.0
b  2.0  2.0
c  3.0  3.0
d  NaN  4.0


You can specify custom index or columns. If you pass `columns`, it overrides the dict’s keys.

In [18]:
pd.DataFrame(d, index=["d", "b", "a"], columns=["two", "three"])

,two,three
d,4.0,NaN
b,2.0,NaN
a,1.0,NaN


Access labels:

In [19]:
df.index     # Index(['a', 'b', 'c', 'd'])
df.columns   # Index(['one', 'two'])

Index(['one', 'two'], dtype='object')

#### From a dict of lists or ndarrays

All arrays must be the same length.

In [20]:
d = {"one": [1.0, 2.0, 3.0, 4.0],
     "two": [4.0, 3.0, 2.0, 1.0]}

pd.DataFrame(d, index=["a", "b", "c", "d"])

,one,two
a,1.0,4.0
b,2.0,3.0
c,3.0,2.0
d,4.0,1.0


#### From a Series

If you convert a Series to a DataFrame, it becomes a **single-column** DataFrame.

In [21]:
ser = pd.Series(range(3), index=list("abc"), name="ser")
pd.DataFrame(ser)

,ser
a,0
b,1
c,2


#### `DataFrame.from_records()`

Useful for structured arrays or lists of tuples.

In [22]:
data = np.array([(1, 2., b'Hello'), (2, 3., b'World')],
                dtype=[('A', '<i4'), ('B', '<f4'), ('C', 'S10')])

pd.DataFrame.from_records(data, index="C")

,A,B
C,,
b'Hello',1,2.0
b'World',2,3.0


## Summary

| Input Type       | Example                                | Notes                      |
| ---------------- | -------------------------------------- | -------------------------- |
| Dict of Series   | `pd.DataFrame({'A': s1, 'B': s2})`     | aligns by index            |
| Dict of lists    | `pd.DataFrame({'A':[1,2], 'B':[3,4]})` | arrays must be same length |
| List of dicts    | `[{'a':1}, {'a':2,'b':3}]`             | flexible per-row keys      |
| Structured array | NumPy dtype records                    | supports typed fields      |
| Series           | `pd.DataFrame(s)`                      | single column              |
| Namedtuples      | `[Point(1,2), Point(3,4)]`             | columns from fields        |
| Dataclasses      | `[Point(x=1,y=2)]`                     | same as dicts              |
| from_dict()      | `orient='index'`                       | use keys as rows           |
| from_records()   | structured input                       | allows index by field      |

# Viewing Data in pandas

When working with DataFrames, it’s often useful to **quickly inspect** your data — to see what it looks like, what its structure is, and get an idea of the values inside.

## `head()` and `tail()`

Use `.head()` and `.tail()` to view the **first** or **last** rows of your DataFrame. By default, `.head()` and `.tail()` display 5 rows. You can pass a number to show more or fewer rows, e.g. `df.head(10)`.


In [23]:
np.random.seed(42)

df = pd.DataFrame(
    np.random.randn(60, 4),
    columns=list("ABCD")
)

df.head()

,A,B,C,D
0,0.496714,-0.138264,0.647689,1.523030
1,-0.234153,-0.234137,1.579213,0.767435
2,-0.469474,0.542560,-0.463418,-0.465730
3,0.241962,-1.913280,-1.724918,-0.562288
4,-1.012831,0.314247,-0.908024,-1.412304


In [24]:
df.tail(3)

,A,B,C,D
57,-0.715304,0.679598,-0.730367,0.216459
58,0.045572,-0.651600,2.143944,0.633919
59,-2.025143,0.186454,-0.661786,0.852433


## Viewing Index and Column Labels

You can easily view your DataFrame’s **row labels (index)** and **column labels**.

In [25]:
df.index

RangeIndex(start=0, stop=60, step=1)

In [26]:
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

## Converting to NumPy

If you want just the raw numeric data **without labels**, use `.to_numpy()`. Note:
* NumPy arrays have **one data type** for the entire array.
* pandas DataFrames can have **different data types per column**.
  When converting, pandas finds the **common dtype** that can hold all columns.
  If the data are mixed, `.to_numpy()` will use `dtype=object` and may need to **copy** data.

In [27]:
df.to_numpy()

array([[ 0.49671415, -0.1382643 ,  0.64768854,  1.52302986],
       [-0.23415337, -0.23413696,  1.57921282,  0.76743473],
       [-0.46947439,  0.54256004, -0.46341769, -0.46572975],
       [ 0.24196227, -1.91328024, -1.72491783, -0.56228753],
       [-1.01283112,  0.31424733, -0.90802408, -1.4123037 ],
       [ 1.46564877, -0.2257763 ,  0.0675282 , -1.42474819],
       [-0.54438272,  0.11092259, -1.15099358,  0.37569802],
       [-0.60063869, -0.29169375, -0.60170661,  1.85227818],
       [-0.01349722, -1.05771093,  0.82254491, -1.22084365],
       [ 0.2088636 , -1.95967012, -1.32818605,  0.19686124],
       [ 0.73846658,  0.17136828, -0.11564828, -0.3011037 ],
       [-1.47852199, -0.71984421, -0.46063877,  1.05712223],
       [ 0.34361829, -1.76304016,  0.32408397, -0.38508228],
       [-0.676922  ,  0.61167629,  1.03099952,  0.93128012],
       [-0.83921752, -0.30921238,  0.33126343,  0.97554513],
       [-0.47917424, -0.18565898, -1.10633497, -1.19620662],
       [ 0.81252582,  1.

### Example: Mixed Data Types

In [28]:
df = pd.DataFrame(np.random.randn(3, 3), columns=list("ABC"))
arr = df.to_numpy(copy=False)
print("Shares memory with df?:", np.shares_memory(arr, df.to_numpy()))

df2 = pd.DataFrame({
    "A": [1.0, 2.0, 3.0],
    "B": ["x", "y", "z"],
})
arr2 = df2.to_numpy(copy=False)
print("Shares memory with df?:", np.shares_memory(arr2, df2.to_numpy()))

df3 = pd.DataFrame(np.random.randn(3, 3), columns=list("ABC"))
arr3 = df3.to_numpy(copy=True)
print("Shares memory with df?:", np.shares_memory(arr3, df3.to_numpy()))


Shares memory with df?: True
Shares memory with df?: False
Shares memory with df?: False


**NumPy Arrays with `dtype=object`**

When a NumPy array has `dtype=object`, it means that **each element is a generic Python object**, not a fixed numeric type. This allows the array to contain mixed data — for example, integers, floats, strings, lists, or even `None`.

NumPy’s normal fast operations (implemented in C) **cannot be used** when the data type is `object`, because NumPy doesn’t know how to, for example, add a string and a number.

Instead, operations are performed **element by element in pure Python** using the standard Python operators.

**Example:**

In [29]:
a = np.array([1, 2, 3], dtype=object)
b = np.array([10, 20, 30], dtype=object)
print(a + b)  # [11 22 33]

try:
   a = np.array([1, "2", 3], dtype=object)
   b = np.array([10, 20, 30], dtype=object)
   print(a + b)  # TypeError
except TypeError as e:
   print("TypeError:", e)

[11 22 33]
TypeError: can only concatenate str (not "int") to str


Key Limitations

- **No vectorization:** operations are slow, executed in Python loops.  
- **No guaranteed numeric behavior:** e.g., `"2" + "3"` → `'23'`, not `5`.  
- **Most NumPy math functions won’t work** (`np.mean`, `np.sin`, etc.).  
- **Broadcasting may fail** or behave inconsistently.

## `describe()`

`.describe()` generates a **statistical summary** of numeric columns — count, mean, std, min, max, and quartiles.

In [30]:
df = pd.DataFrame(
    np.random.randn(60, 4),
    columns=list("ABCD")
)

df.describe()

,A,B,C,D
count,60.000000,60.000000,60.000000,60.000000
mean,0.142207,-0.098821,0.040709,-0.002163
std,1.154115,0.942409,0.969861,0.867430
min,-3.241267,-2.301921,-1.713135,-1.952088
25%,-0.536106,-0.796729,-0.757046,-0.599379
50%,0.146947,-0.109632,0.028646,-0.011231
75%,0.853427,0.593830,0.684001,0.624224
max,3.078881,2.092387,2.133033,1.831459


# Selection and Setting in pandas

When working with DataFrames, selecting and assigning data are essential operations. pandas offers **intuitive Python-like indexing** and several **optimized access methods** for better performance and clarity.

## General Note

While using standard Python/NumPy indexing (e.g., `df[0:3]`, `df["A"]`) is fine for interactive work,
for **production code**, prefer pandas’ optimized accessors:

* `df.at[]` – fast label-based scalar access
* `df.iat[]` – fast position-based scalar access
* `df.loc[]` – label-based selection
* `df.iloc[]` – position-based selection

## Basic Selection with `[]` (Getitem)

* `df["A"]` → selects a **single column** as a Series.
* `df[0:3]` → selects **rows by position slice** (like Python slicing).
* `df["20130102":"20130104"]` → selects **rows by label range** (inclusive on both ends).

In [31]:
dates = pd.date_range("2013-01-01", periods=6)
df = pd.DataFrame(
    np.random.randn(6, 4),
    index=dates,
    columns=list("ABCD")
)
print(df)

                   A         B         C         D
2013-01-01 -0.875618 -1.382800  0.926178  1.909417
2013-01-02 -1.398568  0.562969 -0.650643 -0.487125
2013-01-03 -0.592394 -0.863991  0.048522 -0.830950
2013-01-04  0.270457 -0.050238 -0.238948 -0.907564
2013-01-05 -0.576771  0.755391  0.500917 -0.977555
2013-01-06  0.099332  0.751387 -1.669405  0.543360


In [32]:
print("df[\"A\"]\n", df["A"], end="\n\n")
print("df[0:3]\n", df[0:3], end="\n\n")
print("df[\"20130102\":\"20130104\"]\n", df["20130102":"20130104"])

df["A"]
 2013-01-01   -0.875618
2013-01-02   -1.398568
2013-01-03   -0.592394
2013-01-04    0.270457
2013-01-05   -0.576771
2013-01-06    0.099332
Freq: D, Name: A, dtype: float64

df[0:3]
                    A         B         C         D
2013-01-01 -0.875618 -1.382800  0.926178  1.909417
2013-01-02 -1.398568  0.562969 -0.650643 -0.487125
2013-01-03 -0.592394 -0.863991  0.048522 -0.830950

df["20130102":"20130104"]
                    A         B         C         D
2013-01-02 -1.398568  0.562969 -0.650643 -0.487125
2013-01-03 -0.592394 -0.863991  0.048522 -0.830950
2013-01-04  0.270457 -0.050238 -0.238948 -0.907564


## Selection by Label (`.loc[]` and `.at[]`)

Label-based selection uses the **index labels** (not integer positions).

### Example

In [33]:
# Single row by label
print("Single row by label (df.loc[dates[0]]):\n", df.loc[dates[0]], "\n")

# All rows, only columns A and B
print("All rows, only columns A and B (df.loc[:, ['A', 'B']]):\n", df.loc[:, ["A", "B"]], "\n")

# Range of rows by date labels
print("Range of rows by date labels (df.loc['20130102':'20130104']):\n", df.loc["20130102":"20130104"], "\n")

# Single scalar value
print("Single scalar value (df.loc[dates[0], 'A']):", df.loc[dates[0], "A"], "\n")

# Same as above, faster for single access
print("Single scalar value, fast access (df.at[dates[0], 'A']):", df.at[dates[0], "A"], "\n")

Single row by label (df.loc[dates[0]]):
 A   -0.875618
B   -1.382800
C    0.926178
D    1.909417
Name: 2013-01-01 00:00:00, dtype: float64 

All rows, only columns A and B (df.loc[:, ['A', 'B']]):
                    A         B
2013-01-01 -0.875618 -1.382800
2013-01-02 -1.398568  0.562969
2013-01-03 -0.592394 -0.863991
2013-01-04  0.270457 -0.050238
2013-01-05 -0.576771  0.755391
2013-01-06  0.099332  0.751387 

Range of rows by date labels (df.loc['20130102':'20130104']):
                    A         B         C         D
2013-01-02 -1.398568  0.562969 -0.650643 -0.487125
2013-01-03 -0.592394 -0.863991  0.048522 -0.830950
2013-01-04  0.270457 -0.050238 -0.238948 -0.907564 

Single scalar value (df.loc[dates[0], 'A']): -0.8756182533847572 

Single scalar value, fast access (df.at[dates[0], 'A']): -0.8756182533847572 



## Selection by Position (`.iloc[]` and `.iat[]`)

Position-based selection uses **integer indexes** (like NumPy). `.iloc` slicing works just like NumPy — **end-exclusive**.

### Example

In [34]:
# Row 3
print("Row 3 (df.iloc[3]):\n", df.iloc[3], "\n")

# Rows 3–4, columns 0–1
print("Rows 3–4, columns 0–1 (df.iloc[3:5, 0:2]):\n", df.iloc[3:5, 0:2], "\n")

# Specific rows & columns
print("Specific rows & columns (df.iloc[[1,2,4],[0,2]]):\n", df.iloc[[1, 2, 4], [0, 2]], "\n")

# Single scalar by position
print("Single scalar by position (df.iloc[1,1]):", df.iloc[1, 1], "\n")

# Same as above, faster for single access
print("Single scalar, fast access (df.iat[1,1]):", df.iat[1, 1], "\n")

Row 3 (df.iloc[3]):
 A    0.270457
B   -0.050238
C   -0.238948
D   -0.907564
Name: 2013-01-04 00:00:00, dtype: float64 

Rows 3–4, columns 0–1 (df.iloc[3:5, 0:2]):
                    A         B
2013-01-04  0.270457 -0.050238
2013-01-05 -0.576771  0.755391 

Specific rows & columns (df.iloc[[1,2,4],[0,2]]):
                    A         C
2013-01-02 -1.398568 -0.650643
2013-01-03 -0.592394  0.048522
2013-01-05 -0.576771  0.500917 

Single scalar by position (df.iloc[1,1]): 0.5629692366905709 

Single scalar, fast access (df.iat[1,1]): 0.5629692366905709 



## Boolean Indexing

Use logical conditions to filter data.

In [35]:
print("Rows where column A > 0:\n", df[df["A"] > 0], "\n")

print("Selecting values from a DataFrame where a boolean condition is met:\n", df[df > 0], "\n")

Rows where column A > 0:
                    A         B         C         D
2013-01-04  0.270457 -0.050238 -0.238948 -0.907564
2013-01-06  0.099332  0.751387 -1.669405  0.543360 

Selecting values from a DataFrame where a boolean condition is met:
                    A         B         C         D
2013-01-01       NaN       NaN  0.926178  1.909417
2013-01-02       NaN  0.562969       NaN       NaN
2013-01-03       NaN       NaN  0.048522       NaN
2013-01-04  0.270457       NaN       NaN       NaN
2013-01-05       NaN  0.755391  0.500917       NaN
2013-01-06  0.099332  0.751387       NaN  0.543360 



You can also use `.isin()` for membership filters:

In [36]:
df2 = df.copy()
df2["E"] = ["one", "one", "two", "three", "four", "three"]

# Filter using isin()
print("Filter rows where column E is 'two' or 'four':\n", df2[df2["E"].isin(["two", "four"])], "\n")

Filter rows where column E is 'two' or 'four':
                    A         B         C         D     E
2013-01-03 -0.592394 -0.863991  0.048522 -0.830950   two
2013-01-05 -0.576771  0.755391  0.500917 -0.977555  four 



## Setting Values

You can assign new data to columns or specific cells. Data is aligned by index automatically; missing values become `NaN`.
### Example

In [37]:
# Sample Series for new column
s1 = pd.Series([1, 2, 3, 4, 5, 6], index=dates)

# Add new column, automatically aligned by index
df["F"] = s1
print("DataFrame after adding new column F:\n", df, "\n")

# Set values by label
df.at[dates[0], "A"] = 0
print("After setting df.at[dates[0], 'A'] = 0:\n", df, "\n")

# Set values by position
df.iat[0, 1] = 0
print("After setting df.iat[0, 1] = 0:\n", df, "\n")

# Assign NumPy array to a column
df.loc[:, "D"] = np.array([5] * len(df))
print("After assigning NumPy array to column D:\n", df, "\n")

DataFrame after adding new column F:
                    A         B         C         D  F
2013-01-01 -0.875618 -1.382800  0.926178  1.909417  1
2013-01-02 -1.398568  0.562969 -0.650643 -0.487125  2
2013-01-03 -0.592394 -0.863991  0.048522 -0.830950  3
2013-01-04  0.270457 -0.050238 -0.238948 -0.907564  4
2013-01-05 -0.576771  0.755391  0.500917 -0.977555  5
2013-01-06  0.099332  0.751387 -1.669405  0.543360  6 

After setting df.at[dates[0], 'A'] = 0:
                    A         B         C         D  F
2013-01-01  0.000000 -1.382800  0.926178  1.909417  1
2013-01-02 -1.398568  0.562969 -0.650643 -0.487125  2
2013-01-03 -0.592394 -0.863991  0.048522 -0.830950  3
2013-01-04  0.270457 -0.050238 -0.238948 -0.907564  4
2013-01-05 -0.576771  0.755391  0.500917 -0.977555  5
2013-01-06  0.099332  0.751387 -1.669405  0.543360  6 

After setting df.iat[0, 1] = 0:
                    A         B         C         D  F
2013-01-01  0.000000  0.000000  0.926178  1.909417  1
2013-01-02 -1.398568

### Example: Conditional Setting (Using `where`-like logic)

This replaces all positive values with their negatives — a common pattern in conditional data updates.

In [38]:
df2 = df.copy()
df2[df2 > 0] = -df2
print("Conditional assignment (all positive values negated):\n", df2, "\n")

Conditional assignment (all positive values negated):
                    A         B         C    D  F
2013-01-01  0.000000  0.000000 -0.926178 -5.0 -1
2013-01-02 -1.398568 -0.562969 -0.650643 -5.0 -2
2013-01-03 -0.592394 -0.863991 -0.048522 -5.0 -3
2013-01-04 -0.270457 -0.050238 -0.238948 -5.0 -4
2013-01-05 -0.576771 -0.755391 -0.500917 -5.0 -5
2013-01-06 -0.099332 -0.751387 -1.669405 -5.0 -6 



**Step-by-step:**
1. `df2 > 0` creates a **Boolean mask** where each element is `True` if it’s positive.  
2. `df2[df2 > 0]` selects only those positive cells.  
3. `= -df2` assigns the **negated values** to those selected positions.

Pandas **matches values by index and column labels** (broadcasting with label alignment).  
Therefore, only the cells where the mask was `True` are replaced — all other cells remain unchanged.

Let's see another example:

In [39]:
df = pd.DataFrame({"A": [2, 15, -3, 8]})
df[df["A"] > 10] = 100
print(df)

     A
0    2
1  100
2   -3
3    8


## Summary Table

| Operation Type     | Method                          | Access by         | Returns            | Notes               |
| ------------------ | ------------------------------- | ----------------- | ------------------ | ------------------- |
| Column / Row slice | `[]`                            | label or position | Series / DataFrame | simple selection    |
| Label-based        | `.loc[]`, `.at[]`               | index label       | DataFrame / scalar | endpoints included  |
| Position-based     | `.iloc[]`, `.iat[]`             | integer index     | DataFrame / scalar | NumPy-like behavior |
| Conditional        | boolean mask / `.isin()`        | logical condition | filtered DataFrame | flexible filtering  |
| Setting            | assignment / `.at[]` / `.iat[]` | label or position | modifies data      | index-aligned       |

# Exploring a Student Dataset with pandas

We will explore a [small dataset](https://www.kaggle.com/datasets/saadaliyaseen/analyzing-student-academic-trends) about students’ study habits and exam performance.
We’ll focus on:

* Reading a CSV file
* Using `head()` and `describe()`
* Creating new columns with arithmetic
* Filtering and sorting
* Drawing insights through chained operations

## Step 1: Load the dataset

In [40]:
import pandas as pd

df = pd.read_csv('./data/student_exam_scores.csv')

## Step 2: First look at the data

In [41]:
# Display first few rows
print(df.head(), end="\n\n")

# Display statistical summary
print(df.describe(), end="\n\n")

# Check dataset shape
print('shape: ', df.shape)

  student_id  hours_studied  sleep_hours  attendance_percent  previous_scores  \
0       S001            8.0          8.8                72.1               45   
1       S002            1.3          8.6                60.7               55   
2       S003            4.0          8.2                73.7               86   
3       S004            3.5          4.8                95.1               66   
4       S005            9.1          6.4                89.8               71   

   exam_score  
0        30.2  
1        25.0  
2        35.8  
3        34.0  
4        40.3  

       hours_studied  sleep_hours  attendance_percent  previous_scores  \
count     200.000000   200.000000          200.000000       200.000000   
mean        6.325500     6.622000           74.830000        66.800000   
std         3.227317     1.497138           14.249905        15.663869   
min         1.000000     4.000000           50.300000        40.000000   
25%         3.500000     5.300000           62

## Step 3: Simple selection and inspection

Here, we select only the hours_studied and exam_score columns for a quick look.

In [42]:
df[['student_id', 'hours_studied', 'exam_score']]

,student_id,hours_studied,exam_score
0,S001,8.0,30.2
1,S002,1.3,25.0
2,S003,4.0,35.8
3,S004,3.5,34.0
4,S005,9.1,40.3
...,...,...,...
195,S196,10.5,42.7
196,S197,7.1,40.4
197,S198,1.6,28.2
198,S199,12.0,42.0


## Step 4: Sorting and comparisons

This quickly shows who scored highest in the exam:

In [43]:
df.sort_values('exam_score', ascending=False)

,student_id,hours_studied,sleep_hours,attendance_percent,previous_scores,exam_score
86,S087,12.0,8.2,71.0,95,51.3
83,S084,10.9,5.3,87.7,88,48.9
60,S061,11.9,8.4,53.9,78,48.6
120,S121,9.4,8.3,100.0,78,47.9
71,S072,11.4,5.5,77.1,85,47.9
...,...,...,...,...,...,...
70,S071,3.3,8.1,70.4,40,20.8
183,S184,2.1,6.0,54.3,54,19.4
142,S143,3.9,4.1,55.4,53,19.0
7,S008,2.0,4.3,75.8,55,18.3


Sorting multiple columns helps reveal trade-offs, e.g., more study vs. less sleep:

In [44]:
# Sort by two criteria: hours studied (descending) and sleep (ascending)
df.sort_values(['hours_studied', 'sleep_hours'], ascending=[False, True])

,student_id,hours_studied,sleep_hours,attendance_percent,previous_scores,exam_score
96,S097,12.0,6.1,70.2,56,36.9
198,S199,12.0,7.3,50.5,58,42.0
86,S087,12.0,8.2,71.0,95,51.3
60,S061,11.9,8.4,53.9,78,48.6
150,S151,11.9,8.9,84.5,51,44.1
...,...,...,...,...,...,...
9,S010,1.3,8.9,66.8,70,24.7
126,S127,1.2,5.7,97.5,73,26.8
19,S020,1.1,5.5,53.6,65,17.1
100,S101,1.1,8.5,81.1,92,31.4


## Step 5: Simple arithmetic operations

Creating new columns is often how we generate new insights.

In [45]:
# Calculate difference between current exam and previous score
df['score_improvement'] = df['exam_score'] - df['previous_scores']

# Calculate study efficiency: score per hour studied
df['efficiency'] = df['exam_score'] / df['hours_studied']

# Display both new columns
df[['student_id', 'score_improvement', 'efficiency']]

,student_id,score_improvement,efficiency
0,S001,-14.8,3.775000
1,S002,-30.0,19.230769
2,S003,-50.2,8.950000
3,S004,-32.0,9.714286
4,S005,-30.7,4.428571
...,...,...,...
195,S196,-44.3,4.066667
196,S197,-51.6,5.690141
197,S198,-47.8,17.625000
198,S199,-16.0,3.500000


## Step 6: Logical filtering (conditions)

You must use parentheses `()` around each condition when combining them with `&` or `|`.

In [46]:
# Find students who studied more than 8 hours
df[df['hours_studied'] > 8]

,student_id,hours_studied,sleep_hours,attendance_percent,previous_scores,exam_score,score_improvement,efficiency
4,S005,9.1,6.4,89.8,71,40.3,-30.7,4.428571
5,S006,8.4,5.1,58.5,75,35.7,-39.3,4.250000
6,S007,10.8,6.0,54.2,88,37.9,-50.1,3.509259
14,S015,8.1,8.8,60.0,90,41.1,-48.9,5.074074
18,S019,9.9,4.8,92.5,54,35.6,-18.4,3.595960
...,...,...,...,...,...,...,...,...
186,S187,10.5,4.6,73.2,57,36.2,-20.8,3.447619
193,S194,11.2,6.7,89.9,59,46.7,-12.3,4.169643
195,S196,10.5,5.4,94.0,87,42.7,-44.3,4.066667
198,S199,12.0,7.3,50.5,58,42.0,-16.0,3.500000


In [47]:
# Find students who slept less than 6 hours AND have low attendance
df[(df['sleep_hours'] < 6) & (df['attendance_percent'] < 70)]

,student_id,hours_studied,sleep_hours,attendance_percent,previous_scores,exam_score,score_improvement,efficiency
5,S006,8.4,5.1,58.5,75,35.7,-39.3,4.250000
19,S020,1.1,5.5,53.6,65,17.1,-47.9,15.545455
30,S031,9.9,4.4,55.3,67,34.5,-32.5,3.484848
35,S036,7.1,4.6,52.5,84,32.0,-52.0,4.507042
62,S063,7.1,5.8,63.6,46,27.1,-18.9,3.816901
63,S064,8.5,4.8,66.0,93,36.1,-56.9,4.247059
69,S070,3.9,4.0,53.2,59,23.9,-35.1,6.128205
81,S082,3.9,5.0,50.3,80,32.4,-47.6,8.307692
90,S091,2.2,4.0,61.0,70,27.3,-42.7,12.409091
93,S094,5.6,5.3,51.5,61,29.1,-31.9,5.196429


In [48]:
# Find students who scored above 35 but studied less than 5 hours
df[(df['exam_score'] > 35) & (df['hours_studied'] < 5)]

,student_id,hours_studied,sleep_hours,attendance_percent,previous_scores,exam_score,score_improvement,efficiency
2,S003,4.0,8.2,73.7,86,35.8,-50.2,8.950000
23,S024,2.7,7.7,84.8,82,35.9,-46.1,13.296296
42,S043,3.5,6.6,79.8,86,37.2,-48.8,10.628571
56,S057,2.9,4.4,88.3,86,35.2,-50.8,12.137931
125,S126,4.6,7.7,87.7,89,38.6,-50.4,8.391304


## Step 7: Conditional calculations and nested logic

We can use conditional logic directly inside column creation:

In [49]:
# Create a column labeling performance
df['performance_label'] = df['exam_score'].apply(lambda x: 'High' if x >= 36 else ('Medium' if x >= 30 else 'Low'))

# Label attendance quality
df['attendance_label'] = ['Good' if x >= 75 else 'Poor' for x in df['attendance_percent']]

df[['student_id', 'exam_score', 'performance_label', 'attendance_label']]

,student_id,exam_score,performance_label,attendance_label
0,S001,30.2,Medium,Poor
1,S002,25.0,Low,Poor
2,S003,35.8,Medium,Poor
3,S004,34.0,Medium,Good
4,S005,40.3,High,Good
...,...,...,...,...
195,S196,42.7,High,Good
196,S197,40.4,High,Good
197,S198,28.2,Low,Poor
198,S199,42.0,High,Poor


## Step 8: Summary statistics

Even simple filtering gives insight into correlations between attendance and exam results:

In [50]:
high_attendance = df[df['attendance_percent'] >= 75]['exam_score'].mean()
low_attendance = df[df['attendance_percent'] < 75]['exam_score'].mean()

print("Average exam score (attendance ≥ 75%):", round(high_attendance, 2))
print("Average exam score (attendance < 75%):", round(low_attendance, 2))

Average exam score (attendance ≥ 75%): 34.96
Average exam score (attendance < 75%): 32.93


## Step 9: Chained operations (filter → sort → select)

Chaining multiple commands in one line helps express complex logic clearly.

In [51]:
# Find top 3 students who sleep the least but still scored above 35
df[df['exam_score'] > 35].sort_values('sleep_hours').head(3)[['student_id', 'sleep_hours', 'exam_score']]

,student_id,sleep_hours,exam_score
97,S098,4.0,36.0
24,S025,4.3,39.2
98,S099,4.4,39.9


## Step 10: Combine multiple conditions and calculated columns

In [52]:
# Create a weighted score: 
# 50% exam, 30% previous_scores, 20% attendance
df['weighted_score'] = (0.5 * df['exam_score']) + (0.3 * df['previous_scores']) + (0.2 * df['attendance_percent'])

# Select students with weighted_score > 50
df[df['weighted_score'] > 50][['student_id', 'weighted_score']]

,student_id,weighted_score
2,S003,58.44
3,S004,55.82
4,S005,59.41
5,S006,52.05
6,S007,56.19
...,...,...
193,S194,59.03
194,S195,53.70
195,S196,66.25
196,S197,64.82


## Step 12: Create logical flags (True/False columns)

In [53]:
# Students who are high achievers: exam_score ≥ 36 AND attendance ≥ 80
df['is_high_achiever'] = (df['exam_score'] >= 36) & (df['attendance_percent'] >= 80)

# Students who might be at risk: exam_score < 30 OR attendance < 60
df['is_at_risk'] = (df['exam_score'] < 30) | (df['attendance_percent'] < 60)

df[['student_id','is_high_achiever','is_at_risk']]

,student_id,is_high_achiever,is_at_risk
0,S001,False,False
1,S002,False,True
2,S003,False,False
3,S004,False,False
4,S005,True,False
...,...,...,...
195,S196,True,False
196,S197,True,False
197,S198,False,True
198,S199,False,True


## Step 13: Filtering with `.query()`

The `DataFrame.query()` method lets you filter rows using a **string-based condition**, similar to how you might write a SQL or logical sentence.
This makes your code cleaner and easier to read.

These returns all students who scored above 35:

### Example 1: Simple filtering

In [54]:
df.query("exam_score > 35")

,student_id,hours_studied,sleep_hours,attendance_percent,previous_scores,exam_score,score_improvement,efficiency,performance_label,attendance_label,weighted_score,is_high_achiever,is_at_risk
2,S003,4.0,8.2,73.7,86,35.8,-50.2,8.950000,Medium,Poor,58.44,False,False
4,S005,9.1,6.4,89.8,71,40.3,-30.7,4.428571,High,Good,59.41,True,False
5,S006,8.4,5.1,58.5,75,35.7,-39.3,4.250000,Medium,Poor,52.05,False,True
6,S007,10.8,6.0,54.2,88,37.9,-50.1,3.509259,High,Poor,56.19,False,True
11,S012,6.6,7.9,87.6,85,35.1,-49.9,5.318182,Medium,Good,60.57,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,S194,11.2,6.7,89.9,59,46.7,-12.3,4.169643,High,Good,59.03,True,False
195,S196,10.5,5.4,94.0,87,42.7,-44.3,4.066667,High,Good,66.25,True,False
196,S197,7.1,6.1,85.1,92,40.4,-51.6,5.690141,High,Good,64.82,True,False
198,S199,12.0,7.3,50.5,58,42.0,-16.0,3.500000,High,Poor,48.50,False,True


### Example 2: Combine multiple conditions

In [55]:
df.query("hours_studied > 5 and sleep_hours >= 6.5")

,student_id,hours_studied,sleep_hours,attendance_percent,previous_scores,exam_score,score_improvement,efficiency,performance_label,attendance_label,weighted_score,is_high_achiever,is_at_risk
0,S001,8.0,8.8,72.1,45,30.2,-14.8,3.775000,Medium,Poor,43.02,False,False
11,S012,6.6,7.9,87.6,85,35.1,-49.9,5.318182,Medium,Good,60.57,False,False
14,S015,8.1,8.8,60.0,90,41.1,-48.9,5.074074,High,Poor,59.55,False,False
15,S016,7.0,9.0,51.2,41,34.1,-6.9,4.871429,Medium,Poor,39.59,False,True
17,S018,7.5,7.6,73.8,58,36.3,-21.7,4.840000,High,Poor,50.31,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,S181,9.0,7.6,63.4,67,38.4,-28.6,4.266667,High,Poor,51.98,False,False
189,S190,5.9,7.4,51.4,44,28.0,-16.0,4.745763,Low,Poor,37.48,False,True
190,S191,5.6,8.5,92.5,43,32.2,-10.8,5.750000,Medium,Good,47.50,False,False
193,S194,11.2,6.7,89.9,59,46.7,-12.3,4.169643,High,Good,59.03,True,False


This is equivalent to:

In [56]:
df[(df['hours_studied'] > 5) & (df['sleep_hours'] >= 6.5)]


,student_id,hours_studied,sleep_hours,attendance_percent,previous_scores,exam_score,score_improvement,efficiency,performance_label,attendance_label,weighted_score,is_high_achiever,is_at_risk
0,S001,8.0,8.8,72.1,45,30.2,-14.8,3.775000,Medium,Poor,43.02,False,False
11,S012,6.6,7.9,87.6,85,35.1,-49.9,5.318182,Medium,Good,60.57,False,False
14,S015,8.1,8.8,60.0,90,41.1,-48.9,5.074074,High,Poor,59.55,False,False
15,S016,7.0,9.0,51.2,41,34.1,-6.9,4.871429,Medium,Poor,39.59,False,True
17,S018,7.5,7.6,73.8,58,36.3,-21.7,4.840000,High,Poor,50.31,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,S181,9.0,7.6,63.4,67,38.4,-28.6,4.266667,High,Poor,51.98,False,False
189,S190,5.9,7.4,51.4,44,28.0,-16.0,4.745763,Low,Poor,37.48,False,True
190,S191,5.6,8.5,92.5,43,32.2,-10.8,5.750000,Medium,Good,47.50,False,False
193,S194,11.2,6.7,89.9,59,46.7,-12.3,4.169643,High,Good,59.03,True,False


…but shorter and easier to read.

### Example 3: Using OR (|)

Finds students with poor attendance **or** low exam performance.

In [57]:
df.query("attendance_percent < 60 or exam_score < 30")

,student_id,hours_studied,sleep_hours,attendance_percent,previous_scores,exam_score,score_improvement,efficiency,performance_label,attendance_label,weighted_score,is_high_achiever,is_at_risk
1,S002,1.3,8.6,60.7,55,25.0,-30.0,19.230769,Low,Poor,41.14,False,True
5,S006,8.4,5.1,58.5,75,35.7,-39.3,4.250000,Medium,Poor,52.05,False,True
6,S007,10.8,6.0,54.2,88,37.9,-50.1,3.509259,High,Poor,56.19,False,True
7,S008,2.0,4.3,75.8,55,18.3,-36.7,9.150000,Low,Good,40.81,False,True
9,S010,1.3,8.9,66.8,70,24.7,-45.3,19.000000,Low,Poor,46.71,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,S190,5.9,7.4,51.4,44,28.0,-16.0,4.745763,Low,Poor,37.48,False,True
191,S192,4.1,7.1,59.1,50,30.7,-19.3,7.487805,Medium,Poor,42.17,False,True
192,S193,3.7,5.5,60.6,90,28.8,-61.2,7.783784,Low,Poor,53.52,False,True
197,S198,1.6,6.9,63.8,76,28.2,-47.8,17.625000,Low,Poor,49.66,False,True


### Example 4: Using variables inside query

The `@` symbol allows you to use Python variables inside the query string.

In [58]:
threshold = 35
df.query("exam_score > @threshold")

,student_id,hours_studied,sleep_hours,attendance_percent,previous_scores,exam_score,score_improvement,efficiency,performance_label,attendance_label,weighted_score,is_high_achiever,is_at_risk
2,S003,4.0,8.2,73.7,86,35.8,-50.2,8.950000,Medium,Poor,58.44,False,False
4,S005,9.1,6.4,89.8,71,40.3,-30.7,4.428571,High,Good,59.41,True,False
5,S006,8.4,5.1,58.5,75,35.7,-39.3,4.250000,Medium,Poor,52.05,False,True
6,S007,10.8,6.0,54.2,88,37.9,-50.1,3.509259,High,Poor,56.19,False,True
11,S012,6.6,7.9,87.6,85,35.1,-49.9,5.318182,Medium,Good,60.57,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,S194,11.2,6.7,89.9,59,46.7,-12.3,4.169643,High,Good,59.03,True,False
195,S196,10.5,5.4,94.0,87,42.7,-44.3,4.066667,High,Good,66.25,True,False
196,S197,7.1,6.1,85.1,92,40.4,-51.6,5.690141,High,Good,64.82,True,False
198,S199,12.0,7.3,50.5,58,42.0,-16.0,3.500000,High,Poor,48.50,False,True


### Example 5: Chained query → sort → select

In [59]:
df.query("sleep_hours < 6.5 and exam_score > 30") \
  .sort_values('exam_score', ascending=False)[['student_id', 'sleep_hours', 'exam_score']]

,student_id,sleep_hours,exam_score
83,S084,5.3,48.9
71,S072,5.5,47.9
53,S054,6.3,46.4
174,S175,5.8,44.8
182,S183,5.5,44.1
...,...,...,...
47,S048,6.0,31.1
177,S178,5.7,31.0
49,S050,5.5,30.6
166,S167,5.3,30.3
